In [ ]:
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 51.8 MB/s eta 0:00:00


In [ ]:
# ===============================================================
#  Notebook-Friendly Code for Generating Protonated Molecules
# ===============================================================

# Import necessary libraries
import copy
import pandas as pd
import os
from rdkit import Chem
from datetime import datetime

# This is the core scientific function, it remains unchanged.
def generate_protonated_forms(smiles):
    """
    Generates all chemically unique, singly-protonated forms of a molecule.

    Args:
        smiles (str): The SMILES string of the molecule to protonate.

    Returns:
        list: A list of dictionaries, where each dictionary contains information
              about a unique protonated form. Returns an empty list on error.
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol:
            print(f"Warning: Could not create molecule from SMILES: {smiles}")
            return []

        protonated_molecules = []

        # Iterate over atoms that can be protonated (N, O, S, P)
        for atom in mol.GetAtoms():
            if atom.GetSymbol() in ['N', 'O', 'S', 'P'] and atom.GetFormalCharge() == 0:
                try:
                    mol_copy = copy.deepcopy(mol)
                    atom_copy = mol_copy.GetAtomWithIdx(atom.GetIdx())

                    # Protonate the atom by increasing its formal charge
                    atom_copy.SetFormalCharge(1)

                    # Update hydrogens and sanitize the molecule to check for validity
                    # RDKit will raise an exception if the protonation is invalid (e.g., hypervalency)
                    Chem.SanitizeMol(mol_copy)

                    # Add explicit hydrogens to get the correct SMILES representation
                    mol_with_hs = Chem.AddHs(mol_copy)

                    # Generate a canonical SMILES to handle duplicates
                    prot_smiles = Chem.MolToSmiles(mol_with_hs, canonical=True)

                    protonated_molecules.append({
                        'protonated_smiles': prot_smiles,
                        'protonation_site_index': atom.GetIdx(),
                        'protonation_element': atom.GetSymbol()
                    })

                except Exception as e:
                    # This site cannot be protonated (e.g., would lead to an invalid chemical state)
                    # print(f"Skipping invalid protonation at index {atom.GetIdx()} for {smiles}: {e}")
                    continue

        # Filter out duplicates that result in the same SMILES (e.g., protonating symmetric atoms)
        unique_molecules = []
        seen_smiles = set()
        for mol_data in protonated_molecules:
            if mol_data['protonated_smiles'] not in seen_smiles:
                unique_molecules.append(mol_data)
                seen_smiles.add(mol_data['protonated_smiles'])

        return unique_molecules

    except Exception as e:
        print(f"Error processing SMILES {smiles}: {e}")
        return []

# This helper function is also great as is.
def process_csv_to_df(input_file, smiles_column='smiles'):
    """
    Reads a CSV, finds protonated forms for each SMILES, and returns a DataFrame.

    Args:
        input_file (str): Path to the input CSV file.
        smiles_column (str): Name of the column containing SMILES strings.

    Returns:
        pandas.DataFrame: A DataFrame containing all generated protonated forms.
    """
    df = pd.read_csv(input_file)

    if smiles_column not in df.columns:
        raise ValueError(f"Column '{smiles_column}' not found in {input_file}")

    all_results = []

    print(f"Processing {len(df)} molecules from {input_file}...")
    for idx, row in df.iterrows():
        smiles = row[smiles_column]
        protonated_forms = generate_protonated_forms(smiles)

        for form in protonated_forms:
            result = {
                'original_index': idx,
                'original_smiles': smiles,
                'protonated_smiles': form['protonated_smiles'],
                'protonation_site_index': form['protonation_site_index'],
                'protonation_element': form['protonation_element']
            }
            # You can also copy over other columns from the original dataframe if needed
            # for col in df.columns:
            #     if col != smiles_column:
            #         result[f'original_{col}'] = row[col]
            all_results.append(result)

    print("Processing complete.")
    return pd.DataFrame(all_results)


# ===================================================================
#  New Notebook-Friendly Wrapper Functions (replaces main())
# ===================================================================

def process_single_smiles(smiles, output_path=None):
    """
    Generates protonated forms for a single SMILES string and optionally saves to CSV.

    Args:
        smiles (str): The SMILES string to process.
        output_path (str, optional): If provided, saves the results to this CSV file path.

    Returns:
        pandas.DataFrame: A DataFrame with the protonated forms.
    """
    print(f"Input SMILES: {smiles}")
    protonated_forms = generate_protonated_forms(smiles)

    if not protonated_forms:
        print("No valid protonated forms generated.")
        return pd.DataFrame()

    results = []
    print(f"Generated {len(protonated_forms)} unique protonated forms:")
    for i, form in enumerate(protonated_forms, 1):
        print(f"{i}. Protonate {form['protonation_element']} at index {form['protonation_site_index']}: {form['protonated_smiles']}")
        results.append({
            'original_smiles': smiles,
            'protonated_smiles': form['protonated_smiles'],
            'protonation_site_index': form['protonation_site_index'],
            'protonation_element': form['protonation_element']
        })

    df = pd.DataFrame(results)

    if output_path:
        df.to_csv(output_path, index=False)
        print(f"\nResults saved to: {output_path}")

    return df

def process_smiles_file(input_path, smiles_column='smiles', output_path=None):
    """
    Processes a CSV file of SMILES strings and optionally saves the results to a new CSV.

    Args:
        input_path (str): The path to the input CSV file.
        smiles_column (str, optional): The name of the SMILES column. Defaults to 'smiles'.
        output_path (str, optional): If provided, saves the results to this CSV file path.
                                     If not, a default name will be generated.

    Returns:
        pandas.DataFrame: A DataFrame with all the protonated forms.
    """
    if not os.path.exists(input_path):
        print(f"Error: Input file not found at {input_path}")
        return pd.DataFrame()

    results_df = process_csv_to_df(input_path, smiles_column)

    if len(results_df) == 0:
        print("No protonated forms generated from the file.")
        return results_df

    # Handle saving the file
    if output_path:
        # Use the user-provided path
        final_output_path = output_path
    else:
        # Create a default timestamped filename
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        base_name = os.path.splitext(os.path.basename(input_path))[0]
        final_output_path = f"protonated_{base_name}_{timestamp}.csv"

    results_df.to_csv(final_output_path, index=False)

    # Print summary statistics
    print(f"\n--- Summary ---")
    print(f"Processed {results_df['original_smiles'].nunique()} unique molecules from the input file.")
    print(f"Generated a total of {len(results_df)} protonated forms.")
    print(f"Results saved to: {final_output_path}")

    element_counts = results_df['protonation_element'].value_counts()
    print("\nProtonation sites by element:")
    for element, count in element_counts.items():
        print(f"  {element}: {count}")

    return results_df

In [ ]:
# ===============================================================
#  Setup and Core Functions
# ===============================================================

import copy
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from IPython.display import display
import ipywidgets as widgets

# --- Core scientific function (from previous script) ---
def generate_protonated_forms(smiles):
    """
    Generates all chemically unique, singly-protonated forms of a molecule.
    """
    try:
        mol = Chem.MolFromSmiles(smiles)
        if not mol:
            # print(f"Warning: Could not create molecule from SMILES: {smiles}")
            return []

        protonated_molecules = []
        for atom in mol.GetAtoms():
            if atom.GetSymbol() in ['N', 'O', 'S', 'P'] and atom.GetFormalCharge() == 0:
                try:
                    mol_copy = copy.deepcopy(mol)
                    atom_copy = mol_copy.GetAtomWithIdx(atom.GetIdx())
                    atom_copy.SetFormalCharge(1)
                    Chem.SanitizeMol(mol_copy)
                    mol_with_hs = Chem.AddHs(mol_copy)
                    prot_smiles = Chem.MolToSmiles(mol_with_hs, canonical=True)

                    protonated_molecules.append({
                        'protonated_smiles': prot_smiles,
                        'protonation_site_index': atom.GetIdx(),
                        'protonation_element': atom.GetSymbol()
                    })
                except Exception:
                    continue

        unique_molecules = []
        seen_smiles = set()
        for mol_data in protonated_molecules:
            if mol_data['protonated_smiles'] not in seen_smiles:
                unique_molecules.append(mol_data)
                seen_smiles.add(mol_data['protonated_smiles'])
        return unique_molecules
    except Exception as e:
        # print(f"Error processing SMILES {smiles}: {e}")
        return []

# --- NEW function to process a DataFrame directly ---
def process_dataframe(df, smiles_column='smiles'):
    """
    Takes a DataFrame, finds protonated forms for each SMILES, and returns a new DataFrame.

    Args:
        df (pandas.DataFrame): The input DataFrame containing molecule data.
        smiles_column (str): Name of the column containing SMILES strings.

    Returns:
        pandas.DataFrame: A new DataFrame containing all generated protonated forms.
    """
    if smiles_column not in df.columns:
        raise ValueError(f"SMILES column '{smiles_column}' not found in the DataFrame.")

    all_results = []
    total_rows = len(df)
    print(f"Processing {total_rows} molecules from the DataFrame...")

    # Iterate over the DataFrame rows
    for idx, row in df.iterrows():
        smiles = row[smiles_column]

        # Keep a copy of all original data
        original_data = row.to_dict()

        protonated_forms = generate_protonated_forms(smiles)

        if not protonated_forms:
            continue

        for form in protonated_forms:
            # Start with the original data
            result = original_data.copy()
            # Add the new protonation-specific data
            result.update({
                'protonated_smiles': form['protonated_smiles'],
                'protonation_site_index': form['protonation_site_index'],
                'protonation_element': form['protonation_element']
            })
            all_results.append(result)

    print("Processing complete.")
    results_df = pd.DataFrame(all_results)

    if len(results_df) > 0:
        print(f"Generated {len(results_df)} total protonated forms from {results_df[smiles_column].nunique()} unique molecules.")
    else:
        print("No protonated forms were generated.")

    return results_df

In [ ]:
# ===============================================================
#  Interactive Visualization (Corrected Version)
# ===============================================================

def visualize_protonation(original_smiles, protonated_df):
    """
    This function is called by the widget. It draws the neutral and
    protonated forms for the selected SMILES with CORRECT highlighting.
    """
    # 1. Draw the Original (Neutral) Molecule
    print("--- Original (Neutral) Molecule ---")
    neutral_mol = Chem.MolFromSmiles(original_smiles)
    if neutral_mol:
        display(neutral_mol)
    else:
        print(f"Could not display neutral molecule for SMILES: {original_smiles}")
        return

    # 2. Find and Draw the Protonated Forms
    print("\n--- Protonated Forms ---")
    forms = protonated_df[protonated_df['smiles'] == original_smiles]

    if forms.empty:
        print("No protonated forms found for this molecule.")
        return

    # Prepare lists for drawing
    mols_to_draw = []
    legends = []
    highlight_atoms_list = []

    for _, row in forms.iterrows():
        prot_mol = Chem.MolFromSmiles(row['protonated_smiles'])
        if prot_mol:
            # --- THIS IS THE FIX ---
            # Instead of using the old index, find the atom with the +1 formal charge
            # in the *current* molecule object (prot_mol).
            highlight_idx = -1
            for atom in prot_mol.GetAtoms():
                if atom.GetFormalCharge() == 1:
                    highlight_idx = atom.GetIdx()
                    break # Found it, no need to check other atoms

            mols_to_draw.append(prot_mol)
            # Clarify in the legend that this is the original index
            legends.append(f"Original Idx: {row['protonation_site_index']} ({row['protonation_element']})")

            # Use the newly found index for highlighting
            if highlight_idx != -1:
                highlight_atoms_list.append([highlight_idx])
            else:
                highlight_atoms_list.append([]) # Append empty list if not found

    # 3. Create and display the grid image
    if mols_to_draw:
        img = Draw.MolsToGridImage(
            mols_to_draw,
            molsPerRow=4,
            subImgSize=(250, 250),
            legends=legends,
            highlightAtomLists=highlight_atoms_list
        )
        display(img)
    else:
        print("Could not generate images for protonated forms.")



In [ ]:
# Load your dataset
try:
    dft_molecules_dataset = pd.read_csv("DFT_enhanced_256molecules_20250904_140333.csv")
    print("Dataset loaded successfully.")
    display(dft_molecules_dataset.head(3))

    # Process the entire DataFrame to generate all protonated forms
    protonated_results_df = process_dataframe(dft_molecules_dataset, smiles_column='smiles')

    # Display the first few results
    print("\n--- Sample of Protonated Results ---")
    display(protonated_results_df.head())

except FileNotFoundError:
    print("Error: The file 'DFT_enhanced_256molecules_20250904_140333.csv' was not found.")
    print("Please make sure you have uploaded the file to your Colab environment.")
    protonated_results_df = pd.DataFrame() # Create an empty df to prevent errors later


# --- Create and display the widget (this part is unchanged) ---
if not protonated_results_df.empty:
    # Get a list of unique original SMILES that have protonated forms
    available_smiles = sorted(protonated_results_df['smiles'].unique().tolist())

    # Create the dropdown
    smiles_dropdown = widgets.Dropdown(
        options=available_smiles,
        description='Select SMILES:',
        style={'description_width': 'initial'},
        layout={'width': 'max-content'}
    )

    # Link the dropdown to the visualization function
    interactive_viewer = widgets.interactive_output(
        visualize_protonation,
        {
            'original_smiles': smiles_dropdown,
            'protonated_df': widgets.fixed(protonated_results_df)
        }
    )

    print("✅ Interactive viewer is ready. Choose a molecule from the dropdown below.")
    display(smiles_dropdown, interactive_viewer)
else:
    print("\n⚠️ Cannot create viewer because no protonated molecules were generated.")

Dataset loaded successfully.


,original_index,smiles,latent_1,latent_2,latent_3,HBD,HBA,heavy_atoms,MW,has_heteroatoms,...,protonatable_sites,cluster_id,calculation_type,is_cluster_center,distance_to_centroid,dft_representative,distance_to_dft,centroid_latent_1,centroid_latent_2,centroid_latent_3
0,13697,NCCN[C@@H]1CNC[C@H]1O,-2.136919,2.862223,-1.063962,4,4,10,145.206,True,...,4,0,DFT,True,0.040002,728658,0.0,-2.110097,2.839517,-1.083070
1,47871,Cc1noc([C@@H](N)CO)n1,2.607847,0.615091,-2.075723,2,5,10,143.146,True,...,5,1,DFT,True,0.061465,686946,0.0,2.558563,0.609500,-2.039423
2,178762,CCNC(=O)CCOC,-0.636599,-4.092548,1.650663,1,2,9,131.175,True,...,3,2,DFT,True,0.023171,286252,0.0,-0.644349,-4.083014,1.670309


Processing 256 molecules from the DataFrame...


[15:24:32] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 9
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 1 2 3 6 10
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 2 3 4 5 6
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 8 9 10 11 13
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 8 9 10 11 12
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 8 9 10 11 12
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 8 9 10 11 12
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 4 5 6 9 10
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 6 7 8 9 10
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 4 5 6 7 8
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 11
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 1 2 3 7 11
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 1 2 3 11 12
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 8 9 10 11 1

Processing complete.
Generated 834 total protonated forms from 256 unique molecules.

--- Sample of Protonated Results ---


[15:24:32] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 11
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 2 3 4 12 13
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 1 2 3 12 13
[15:24:32] Explicit valence for atom # 1 S, 6, is greater than permitted
[15:24:32] Explicit valence for atom # 4 S, 6, is greater than permitted
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 1 2 3 4 5 9 10 11 12
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 0 1 2 3 12
[15:24:32] Explicit valence for atom # 4 S, 6, is greater than permitted
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 4 5 6 11 12
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 12
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 9 10 11 12 13
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 5 6 7 8 9
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 8 9 10 11 12
[15:24:32] Can't kekulize mol.  Unkekulized atoms: 7 8 9 10 13
[15:24:32] Can

,original_index,smiles,latent_1,latent_2,latent_3,HBD,HBA,heavy_atoms,MW,has_heteroatoms,...,is_cluster_center,distance_to_centroid,dft_representative,distance_to_dft,centroid_latent_1,centroid_latent_2,centroid_latent_3,protonated_smiles,protonation_site_index,protonation_element
0,13697,NCCN[C@@H]1CNC[C@H]1O,-2.136919,2.862223,-1.063962,4,4,10,145.206,True,...,True,0.040002,728658,0.0,-2.110097,2.839517,-1.083070,[H]O[C@]1([H])C([H])([H])N([H])C([H])([H])[C@@...,0,N
1,13697,NCCN[C@@H]1CNC[C@H]1O,-2.136919,2.862223,-1.063962,4,4,10,145.206,True,...,True,0.040002,728658,0.0,-2.110097,2.839517,-1.083070,[H]O[C@]1([H])C([H])([H])N([H])C([H])([H])[C@@...,3,N
2,13697,NCCN[C@@H]1CNC[C@H]1O,-2.136919,2.862223,-1.063962,4,4,10,145.206,True,...,True,0.040002,728658,0.0,-2.110097,2.839517,-1.083070,[H]O[C@]1([H])C([H])([H])[N+]([H])([H])C([H])(...,6,N
3,13697,NCCN[C@@H]1CNC[C@H]1O,-2.136919,2.862223,-1.063962,4,4,10,145.206,True,...,True,0.040002,728658,0.0,-2.110097,2.839517,-1.083070,[H]N([H])C([H])([H])C([H])([H])N([H])[C@]1([H]...,9,O
4,47871,Cc1noc([C@@H](N)CO)n1,2.607847,0.615091,-2.075723,2,5,10,143.146,True,...,True,0.061465,686946,0.0,2.558563,0.609500,-2.039423,[H]OC([H])([H])[C@@]([H])(c1nc(C([H])([H])[H])...,2,N


✅ Interactive viewer is ready. Choose a molecule from the dropdown below.


Dropdown(description='Select SMILES:', layout=Layout(width='max-content'), options=('C#CCCCCN[C@@H](C=C)C(=O)O…

Output()

# Finding successful molecules

# Do all automatically

In [ ]:

# Python standard libraries
import json
import zipfile
from pathlib import Path

# Data handling and visualization
import pandas as pd
from IPython.display import display

# Google Colab specific
from google.colab import files

# Interactive widgets
import ipywidgets as widgets

# RDKit for chemistry
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit import RDLogger

# Suppress RDKit warnings for cleaner output
RDLogger.DisableLog('rdApp.*')

print("✅ Setup complete. All libraries are ready.")

✅ Setup complete. All libraries are ready.


In [ ]:
# ==============================================================================
# SECTION 2 (REVISED): ROBUST UPLOAD AND DATA PREPARATION
# This version is immune to the __MACOSX issue.
# ==============================================================================

# --- Step 2.1: Upload your zip file ---
print("Please select the ZIP file containing your results folder...")
uploaded = files.upload()

# --- Step 2.2: Unzip the file and load data ---
if not uploaded:
    print("\n❌ No file was uploaded. Please run the cell again.")
else:
    zip_filename = next(iter(uploaded))
    extract_dir = Path("unzipped_pm7_results")

    # Clean up previous runs if they exist
    if extract_dir.exists():
        import shutil
        shutil.rmtree(extract_dir)

    extract_dir.mkdir(exist_ok=True, parents=True)

    print(f"\nUnzipping '{zip_filename}' into '{extract_dir}'...")
    with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    # --- Step 2.3: INTELLIGENTLY find the correct results directory ---
    results_folder_path = None
    unzipped_contents = list(extract_dir.iterdir())

    # Search for the first valid directory that is NOT a macOS artifact
    for item in unzipped_contents:
        if item.is_dir() and item.name != "__MACOSX":
            results_folder_path = item
            break # Found it, stop searching

    # If the loop didn't find a subdirectory, assume files are in the top level
    if results_folder_path is None:
        results_folder_path = extract_dir

    print(f"Data folder intelligently identified at: '{results_folder_path}'")

    # --- Step 2.4: Load the core JSON data into memory ---
    try:
        with open(results_folder_path / "protonation_map.json", 'r') as f:
            full_map_data = json.load(f)
        with open(results_folder_path / "molecular_properties.json", 'r') as f:
            properties_data = json.load(f)
        print("\n✅ Successfully loaded 'protonation_map.json' and 'molecular_properties.json'.")
        print("Data is now ready for analysis.")
    except FileNotFoundError:
        print("\n❌ Error: Could not find required JSON files in the identified folder.")
        print("Please check the contents of your zip file.")
        full_map_data, properties_data = {}, {}

Please select the ZIP file containing your results folder...


Saving results_DFT_enhanced_256molecules_20250903_222036.zip to results_DFT_enhanced_256molecules_20250903_222036 (5).zip

Unzipping 'results_DFT_enhanced_256molecules_20250903_222036 (5).zip' into 'unzipped_pm7_results'...
Data folder intelligently identified at: 'unzipped_pm7_results/results_DFT_enhanced_256molecules_20250903_222036'

✅ Successfully loaded 'protonation_map.json' and 'molecular_properties.json'.
Data is now ready for analysis.


In [ ]:
# ==============================================================================
# SECTION 3: ANALYSIS 1 - GENERATE STATISTICAL SUMMARY REPORT
# ==============================================================================

def generate_detailed_summary(results_dir, protonation_map, properties):
    """Analyzes PM7 outputs to provide a detailed statistical summary."""
    print(f"\nGenerating Detailed Summary from: {results_dir}\n")

    total_neutral_molecules = len(protonation_map)
    total_potential_protonations, total_successful_neutrals, total_failed_neutrals = 0, 0, 0
    total_successful_protonations, total_failed_protonations = 0, 0
    neutrals_fully_successful, neutrals_with_some_failures = 0, 0

    for neutral_smiles, all_protonated_forms in protonation_map.items():
        total_potential_protonations += len(all_protonated_forms)
        is_neutral_success = properties.get(neutral_smiles, {}).get("success", False)
        if is_neutral_success: total_successful_neutrals += 1
        else: total_failed_neutrals += 1

        successful_prots_for_this_mol = sum(1 for p_smi in all_protonated_forms if properties.get(p_smi, {}).get("success", False))
        total_successful_protonations += successful_prots_for_this_mol
        total_failed_protonations += (len(all_protonated_forms) - successful_prots_for_this_mol)

        if is_neutral_success and successful_prots_for_this_mol == len(all_protonated_forms):
            neutrals_fully_successful += 1
        else:
            neutrals_with_some_failures += 1

    print("="*50); print("      COMPREHENSIVE CALCULATION SUMMARY REPORT"); print("="*50)
    print(f"\n--- Overall Molecule Counts ---\nTotal Neutral Molecules Analyzed:       {total_neutral_molecules}\nTotal Potential Protonated Forms:       {total_potential_protonations}\nTotal Calculations Attempted:           {total_neutral_molecules + total_potential_protonations}")
    print(f"\n--- Breakdown by Calculation Type ---\nSuccessful Neutral Calculations:        {total_successful_neutrals}\nFailed Neutral Calculations:            {total_failed_neutrals}\n" + "-"*40 + f"\nSuccessful Protonated Calculations:     {total_successful_protonations}\nFailed Protonated Calculations:         {total_failed_protonations}")
    print(f"\n--- Summary by Molecule Family ---\nPerfect Families (Neutral + ALL Prots OK): {neutrals_fully_successful}\nProblematic Families (>= 1 Failure):     {neutrals_with_some_failures}")
    print("="*50)

# --- Run the summary report ---
if 'full_map_data' in locals() and full_map_data:
    generate_detailed_summary(results_folder_path, full_map_data, properties_data)
else:
    print("⚠️ Cannot generate report because data was not loaded in Section 2.")


Generating Detailed Summary from: unzipped_pm7_results/results_DFT_enhanced_256molecules_20250903_222036

      COMPREHENSIVE CALCULATION SUMMARY REPORT

--- Overall Molecule Counts ---
Total Neutral Molecules Analyzed:       256
Total Potential Protonated Forms:       795
Total Calculations Attempted:           1051

--- Breakdown by Calculation Type ---
Successful Neutral Calculations:        249
Failed Neutral Calculations:            7
----------------------------------------
Successful Protonated Calculations:     693
Failed Protonated Calculations:         102

--- Summary by Molecule Family ---
Perfect Families (Neutral + ALL Prots OK): 187
Problematic Families (>= 1 Failure):     69


In [ ]:
# ==============================================================================
# SECTION 4: ANALYSIS 2 - CREATE DFT INPUT FILE
# ==============================================================================

def create_dft_input_csv(results_dir, protonation_map, properties, output_filename="dft_input_pairs.csv"):
    """Creates a two-column CSV file of successful neutral-protonated pairs."""
    print(f"\nSearching for successful pairs in: {results_dir}\n")
    successful_pairs = []

    for neutral_smiles, all_protonated_forms in protonation_map.items():
        if not properties.get(neutral_smiles, {}).get("success", False):
            continue
        for prot_smiles in all_protonated_forms:
            if properties.get(prot_smiles, {}).get("success", False):
                successful_pairs.append({'Neutral_smiles': neutral_smiles, 'Protonated_smiles': prot_smiles})

    if not successful_pairs:
        print("No valid pairs found where both neutral and protonated forms were successful.")
        return pd.DataFrame()

    dft_input_df = pd.DataFrame(successful_pairs)
    output_path = results_dir / output_filename
    dft_input_df.to_csv(output_path, index=False)
    print("--- Summary ---\n✅ CSV file created successfully!")
    print(f"   Saved to: {output_path}\nFound {len(dft_input_df)} successful Neutral-Protonated pairs ready for DFT.")
    return dft_input_df

# --- Run the CSV creation ---
if 'full_map_data' in locals() and full_map_data:
    dft_input_df = create_dft_input_csv(results_folder_path, full_map_data, properties_data)
    if not dft_input_df.empty:
        print("\n--- Preview of the generated CSV data ---")
        display(dft_input_df.head())
else:
    print("⚠️ Cannot create CSV because data was not loaded in Section 2.")


Searching for successful pairs in: unzipped_pm7_results/results_DFT_enhanced_256molecules_20250903_222036

--- Summary ---
✅ CSV file created successfully!
   Saved to: unzipped_pm7_results/results_DFT_enhanced_256molecules_20250903_222036/dft_input_pairs.csv
Found 691 successful Neutral-Protonated pairs ready for DFT.

--- Preview of the generated CSV data ---


,Neutral_smiles,Protonated_smiles
0,C#CC(=O)NCCCC(=O)OC(C)C,[H]C#CC(=[O+][H])N([H])C([H])([H])C([H])([H])C...
1,C#CC(=O)NCCCC(=O)OC(C)C,[H]C#CC(=O)[N+]([H])([H])C([H])([H])C([H])([H]...
2,C#CC(=O)NCCCC(=O)OC(C)C,[H]C#CC(=O)N([H])C([H])([H])C([H])([H])C([H])(...
3,C#CC(=O)NCCCC(=O)OC(C)C,[H]C#CC(=O)N([H])C([H])([H])C([H])([H])C([H])(...
4,C#CC(=O)N[C@@H](C)CCc1ccco1,[H]C#CC(=[O+][H])N([H])[C@@]([H])(C([H])([H])[...


from matplotlib import pyplot as plt
import seaborn as sns
_df_0.groupby('Neutral_smiles').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('Protonated_smiles').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
plt.subplots(figsize=(8, 8))
df_2dhist = pd.DataFrame({
    x_label: grp['Protonated_smiles'].value_counts()
    for x_label, grp in _df_2.groupby('Neutral_smiles')
})
sns.heatmap(df_2dhist, cmap='viridis')
plt.xlabel('Neutral_smiles')
_ = plt.ylabel('Protonated_smiles')

In [ ]:
# ==============================================================================
# SECTION 5: ANALYSIS 3 - INTERACTIVE DEBUGGING VISUALIZER
# ==============================================================================

# --- Step 5.1: Define the Visualization Function ---
def visualize_protonation_status(neutral_smiles, protonation_map, properties):
    """Generates a display showing neutral, successful, and failed forms."""
    print("--- Original (Neutral) Molecule ---")
    display(Chem.MolFromSmiles(neutral_smiles))

    all_protonated_forms = protonation_map.get(neutral_smiles, [])
    successful_forms = [s for s in all_protonated_forms if properties.get(s, {}).get("success", False)]
    unsuccessful_forms = [s for s in all_protonated_forms if not properties.get(s, {}).get("success", False)]

    print(f"\n--- ✅ Successful Protonations ({len(successful_forms)}) ---")
    if successful_forms:
        mols = [Chem.MolFromSmiles(s) for s in successful_forms]
        highlights = [[next((a.GetIdx() for a in m.GetAtoms() if a.GetFormalCharge() == 1), -1)] for m in mols]
        display(Draw.MolsToGridImage(mols, molsPerRow=4, subImgSize=(250, 250), highlightAtomLists=highlights))
    else: print("None found.")

    print(f"\n--- ❌ Unsuccessful / Failed Calculations ({len(unsuccessful_forms)}) ---")
    if unsuccessful_forms:
        mols = [Chem.MolFromSmiles(s) for s in unsuccessful_forms]
        display(Draw.MolsToGridImage(mols, molsPerRow=4, subImgSize=(250, 250)))
    else: print("None found.")

# --- Step 5.2: Filter for Problematic Cases ---
if 'full_map_data' in locals() and full_map_data:
    print("\nScanning for molecules with at least one failed calculation...")
    problematic_smiles = []
    for neutral_smiles, all_protonated_forms in full_map_data.items():
        if not properties_data.get(neutral_smiles, {}).get("success", False):
            problematic_smiles.append(neutral_smiles)
            continue
        for prot_smiles in all_protonated_forms:
            if not properties_data.get(prot_smiles, {}).get("success", False):
                problematic_smiles.append(neutral_smiles)
                break
    print(f"Scan Complete. Found {len(problematic_smiles)} problematic molecules to inspect.")

    # --- Step 5.3: Create and Launch the Widget ---
    if problematic_smiles:
        smiles_dropdown = widgets.Dropdown(options=sorted(problematic_smiles), description='Select Problematic Molecule:', style={'description_width': 'initial'}, layout={'width': 'max-content', 'min_width': '400px'})
        interactive_viewer = widgets.interactive_output(visualize_protonation_status, {'neutral_smiles': smiles_dropdown, 'protonation_map': widgets.fixed(full_map_data), 'properties': widgets.fixed(properties_data)})
        print("\n✅ Interactive viewer for EXCEPTION CASES is ready.")
        display(smiles_dropdown, interactive_viewer)
    else:
        print("\n✅🎉 Congratulations! No problematic molecules were found.")
else:
    print("⚠️ Cannot create visualizer because data was not loaded in Section 2.")


Scanning for molecules with at least one failed calculation...
Scan Complete. Found 69 problematic molecules to inspect.

✅ Interactive viewer for EXCEPTION CASES is ready.


Dropdown(description='Select Problematic Molecule:', layout=Layout(min_width='400px', width='max-content'), op…

Output()

In [ ]:
successful_candidates=pd.read_csv("/content/unzipped_pm7_results/results_DFT_enhanced_256molecules_20250903_222036/dft_input_pairs.csv")